In [1]:
print("hello")

hello


In [2]:
import sys
import os

# ==============================================================================
# ★最優先: bitsandbytes / triton の完全無効化パッチ
# 必ず他のライブラリ(transformers/peft)をインポートする前に実行してください
# ==============================================================================
# 1. bitsandbytes をシステムから隠蔽する (import すると ModuleNotFoundError になるようにする)
sys.modules["bitsandbytes"] = None

# 2. PEFT が bnb を検知する関数を、ライブラリロード前に上書き予約しておく
# (PEFTがまだロードされていないため、sys.modulesへの注入とimport後のパッチ両方を行う)

import peft.import_utils
peft.import_utils.is_bnb_available = lambda: False
peft.import_utils.is_bnb_4bit_available = lambda: False

# ==============================================================================

import torch
import textwrap
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# ==========================================
# 1. 設定
# ==========================================
BASE_MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"
SUBSPACE_PATH = "common_subspace.pt"  # 以前作成したファイル
SAVED_MODEL_PATH = "./saved_models/run-research-hybrid-01" # ★ここを実際の保存パスに変更！

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# ==========================================
# 2. スコア計算用クラス (再掲)
# ==========================================
class ResidualCuriosityRewardModel(torch.nn.Module):
    def __init__(self, path, device):
        super().__init__()
        data = torch.load(path, map_location="cpu", weights_only=False)
        self.register_buffer("basis", data["basis"].to(device, dtype=torch.float16))
        self.register_buffer("mean", data["mean"].to(device, dtype=torch.float16))

    def get_score(self, hidden_states):
        h = hidden_states.to(self.basis.dtype)
        h_centered = h - self.mean
        z_common = h_centered @ self.basis.T
        h_common = z_common @ self.basis
        h_residual = h_centered - h_common
        norms = torch.norm(h_residual, dim=-1)
        return torch.log1p(norms).mean().item()

# ==========================================
# 3. 推論・比較関数
# ==========================================
def compare_models(prompts):
    print(f"Loading Tokenizer: {BASE_MODEL_NAME} ...")
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
    
    # 評価器のロード
    scorer = ResidualCuriosityRewardModel(SUBSPACE_PATH, DEVICE)
    
    # --- A. Base Model 生成 ---
    print("\n>>> Loading BASE MODEL...")
    base_model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL_NAME,
        torch_dtype=torch.float16,
        device_map="auto"
    )
    
    print(">>> Generating with [BASE MODEL]...")
    base_results = generate(base_model, tokenizer, prompts, scorer)
    
    # メモリ節約のためBase Modelを削除
    del base_model
    torch.cuda.empty_cache()
    
    # --- B. Trained Model 生成 ---
    print(f"\n>>> Loading Adapter from {SAVED_MODEL_PATH}...")
    
    # 1. まずベースモデルを再ロード
    base_model_for_peft = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL_NAME,
        torch_dtype=torch.float16,
        device_map="auto"
    )
    
    # 2. アダプターをロード
    # 冒頭のパッチにより bnb/triton は無視されます
    trained_model = PeftModel.from_pretrained(
        base_model_for_peft, 
        SAVED_MODEL_PATH,
        is_trainable=False,
        torch_dtype=torch.float16,
        device_map="auto"
    )
    
    print(">>> Generating with [RDRL TRAINED MODEL]...")
    trained_results = generate(trained_model, tokenizer, prompts, scorer)
    
    # --- 3. 結果表示 ---
    print_comparison(prompts, base_results, trained_results)

def generate(model, tokenizer, prompts, scorer):
    results = []
    for prompt in prompts:
        inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
        
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=128,
                do_sample=True,
                temperature=0.9,
                top_p=0.95,
                repetition_penalty=1.1,
                output_hidden_states=True,
                return_dict_in_generate=True
            )
        
        # テキスト
        gen_text = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
        response = gen_text[len(prompt):]
        
        # スコア計算
        hidden_list = []
        # プロンプト以降のステップを取得
        if len(outputs.hidden_states) > 1:
            for step_data in outputs.hidden_states[1:]:
                # step_data[-1] は最終層
                last_layer = step_data[-1].squeeze(0).squeeze(0)
                hidden_list.append(last_layer)
            
            if hidden_list:
                h_seq = torch.stack(hidden_list, dim=0)
                score = scorer.get_score(h_seq)
            else:
                score = 0.0
        else:
            score = 0.0
            
        results.append({"text": response, "score": score})
    return results

def print_comparison(prompts, base_res, trained_res):
    print("\n" + "="*80)
    print(" CREATIVITY SHOWDOWN: Base vs RDRL")
    print("="*80)
    
    wins = 0
    
    for i, (p, b, t) in enumerate(zip(prompts, base_res, trained_res)):
        print(f"\nPrompt {i+1}: {p.strip()}")
        print("-" * 40)
        
        # Base
        print(f"[Base Model] (Score: {b['score']:.4f})")
        print(textwrap.fill(b['text'].strip(), width=80))
        print("-" * 20)
        
        # Trained
        print(f"[RDRL Model] (Score: {t['score']:.4f})")
        print(textwrap.fill(t['text'].strip(), width=80))
        print("-" * 20)
        
        # 判定
        diff = t['score'] - b['score']
        if diff > 0:
            print(f"🏆 WINNER: RDRL Model (+{diff:.4f})")
            wins += 1
        else:
            print(f"👾 WINNER: Base Model ({diff:.4f})")
            
    print("="*80)
    print(f"Final Result: RDRL Model won {wins} / {len(prompts)} rounds.")

In [4]:
# ==========================================
# 4. テスト用プロンプト (研究アイデア)
# ==========================================
test_prompts = [
    """You are an expert LLM researcher. Propose a novel and concrete research idea about large language models.
Output ONLY in the following format:
Title: <concise LLM research title>
Abstract: <150-220 word abstract>
Draft a research proposal about Mixture of Experts (MoE).""",
    
    """You are an expert LLM researcher. Propose a novel and concrete research idea about large language models.
Output ONLY in the following format:
Title: <concise LLM research title>
Abstract: <150-220 word abstract>
Propose an experiment improving Hallucination Detection with a focus on interpretability.""",
]

In [5]:
compare_models(test_prompts)

Loading Tokenizer: Qwen/Qwen2.5-7B-Instruct ...


`torch_dtype` is deprecated! Use `dtype` instead!



>>> Loading BASE MODEL...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

>>> Generating with [BASE MODEL]...

>>> Loading Adapter from ./saved_models/run-research-hybrid-01...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

>>> Generating with [RDRL TRAINED MODEL]...

 CREATIVITY SHOWDOWN: Base vs RDRL

Prompt 1: You are an expert LLM researcher. Propose a novel and concrete research idea about large language models.
Output ONLY in the following format:
Title: <concise LLM research title>
Abstract: <150-220 word abstract>
Draft a research proposal about Mixture of Experts (MoE).
----------------------------------------
[Base Model] (Score: 5.4805)
Title: Investigating the Performance and Efficiency of Mixture of Experts in
Large Language Models  Abstract: As large language models (LLMs) continue to
grow, they face significant challenges such as increasing inference time,
training costs, and storage requirements. One potential solution is the Mixture
of Experts (MoE), where tasks are distributed across multiple specialized sub-
networks or "experts," each addressing specific aspects of natural language
processing. This study aims to explore how MoE can enhance both the performance
and efficiency of LLMs by